# ACR extraction #

Parse text documents which are based on a template and stored in an excel document

Install dependencies and import the nlp_datau package

In [1]:
import yaml
import re
import sys

!pip install -r ./../requirements.txt
sys.path.append("./../")


  Using cached spacy-2.2.4-cp38-cp38-macosx_10_9_x86_64.whl (10.6 MB)
  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
  Using cached thinc-7.4.0-cp38-cp38-macosx_10_9_x86_64.whl (2.2 MB)
  Using cached srsly-1.0.5-cp38-cp38-macosx_10_9_x86_64.whl (177 kB)
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.2
    Uninstalling srsly-2.4.2:
      Successfully uninstalled srsly-2.4.2
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.6
    Uninstalling catalogue-2.0.6:
      Successfully uninstalled catalogue-2.0.6
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.13
    Uninstalling thinc-8.0.13:
      Successfully uninstalled thinc-8.0.13
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.0
    Uninstalling spacy-3.2.0:
      Successfully uninstalled spacy-3.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

## 1. Extract substring of substring ##

Set variables

In [22]:
path_excel_sheet_in = '/home/jovyan/work/D/ACR-meta/Mammo-20211011-Test-out.xlsx'
# path_excel_sheet_in = '../resources/data_example/test_acr.xlsx'

path_excel_sheet_out = '/home/jovyan/work/D/ACR-meta/Mammo-20211011-Test-out-acr.xlsx'



import pandas
from IPython.core.display import display
df = pandas.read_excel(path_excel_sheet_in, header=0)
display(df)

,Patientnummer,Pat-ID (8tekens),Onderzoeksnummer,Procedure RIS,Datum onderzoek,Datum geboorte,#Leeftijd_Patient_Onderzoek,Patient geslacht,# Procedures,text,R_DATELASTCHANGED,patient_id
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deze patient heeft een doorverwijzing en ACR-4,NaN,NaN
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deze patient heeft een doorverwijzing en ACR-5,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deze patient heeft een doorverwijzing en ACR-6,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deze patient heeft een doorverwijzing en ACR-7,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deze patient heeft een doorverwijzing en ACR-8,NaN,NaN
5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deze patient heeft een doorverwijzing en ACR-9,NaN,NaN


Extract findings

In [18]:
column_source = 'text'

column_names_list = [
    'acr',
    'bevolkingsonderzoek'
]

# https://regex101.com, eerste / kan weg
regex_list = [
    r"(zeer dens.{0,5}klier|ACR-{0,1}(4|vier|IV|d|D)|dense.{0,5}trial)",
    r"(doorver|BVO|bevolking.{0,2}onderzoek)"
]

In [25]:
flags = re.IGNORECASE
def has_match(search_regex, txt):
    if not txt:
        return None
    return bool(re.search(search_regex, txt, flags=flags))

print(has_match(regex_list[0], "ACR-3"))
print(has_match(regex_list[0], "ACR-4"))

False
True


Extract corads

In [26]:
for index, column_target in enumerate(column_names_list):
    search_regex = regex_list[index]
    df[column_target] = df.apply(lambda x:has_match(search_regex=search_regex, txt=x[column_source]), axis=1)

display(df)

,Patientnummer,Pat-ID (8tekens),Onderzoeksnummer,Procedure RIS,Datum onderzoek,Datum geboorte,#Leeftijd_Patient_Onderzoek,Patient geslacht,# Procedures,text,R_DATELASTCHANGED,patient_id,acr,bevolkingsonderzoek
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deze patient heeft een doorverwijzing en ACR-4,NaN,NaN,True,True
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deze patient heeft een doorverwijzing en ACR-5,NaN,NaN,False,True
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deze patient heeft een doorverwijzing en ACR-6,NaN,NaN,False,True
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deze patient heeft een doorverwijzing en ACR-7,NaN,NaN,False,True
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deze patient heeft een doorverwijzing en ACR-8,NaN,NaN,False,True
5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deze patient heeft een doorverwijzing en ACR-9,NaN,NaN,False,True


Run and write to out file

In [20]:
df.to_excel(path_excel_sheet_out)

FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/work/D/ACR-meta/Mammo-20211011-Test-out-acr.xlsx'